## SuSiE Slides

In [ ]:
[SuSiE_slides_1]
input: group_by = 'all'
output: analysis_summary = f'{cwd}/{sumstats_path:bnn}.analysis_summary.md', variants_csv = f'{cwd}/{sumstats_path:bnn}.variants.csv'
R: container=container, expand = "${ }", entrypoint = entrypoint
    # Define the theme string
    theme <- '---'
    theme: base-theme
    style: |
     p {
       font-size: 24px;
       height: 900px;
       margin-top:1cm;
      }
      img {
        height: 70%;
        display: block;
        margin-left: auto;
        margin-right: auto;
      }
      body {
       margin-top: auto;
       margin-bottom: auto;
       font-family: verdana;
      }
    ---    
    '
    text <- ""
    sep <- '\n\n---\n'

    inp <- strsplit("${_input:r}", " ")[[1]]
    inp <- sapply(inp, function(x) paste(head(strsplit(x, "\\.")[[1]], -1), collapse = "."))

    r <- unique(strsplit("${_input:bn}", " ")[[1]])

    num_csets <- numeric()
    region_info <- character()

    variant_info <- list()

    for (reg_i in seq_along(unique(inp))) {

      rid <- unlist(strsplit(r[reg_i], '\\.'))[1]

      text_temp <- ""
      text_temp <- paste0(text_temp, "#\n\n SuSiE RSS ", r[reg_i], " \n")
      text_temp <- paste0(text_temp, "![](", r[reg_i], ".png)", sep, " \n \n")

      rd <- readRDS(substr(each, 2, nchar(each)) + ".rds")

      # find the number of cs in the current region
      if (is.null(rd$sets$cs)) {
        num_csets <- c(num_csets, 0)
      } else {
        num_csets <- c(num_csets, length(rd$sets$cs))
      }
      cat(num_csets, "\n")

      # this will store the indices of all variants that cross the threshold
      ind_p <- which(rd$pip >= ${pip_cutoff})
      sumvars <- 0

      # if we have at least one cs in the current region
      if (num_csets[reg_i] > 0) {
        tbl_header <- "| chr number | pos at highest pip | ref | alt | region id | cs | highest pip |  \n| --- | --- | --- | --- | --- | --- | --- |  \n"

        table <- ""

        sumpips <- 0

        for (cset in names(rd$sets$cs)) {
          print(cset)

          # if we have many variants in the cs
          if (length(rd$sets$cs[[cset]]) > 1) {
            highestpip <- max(rd$pip[rd$sets$cs[[cset]]])
            poswhighestpip <- which.max(rd$pip[rd$sets$cs[[cset]]])

            # we make sure that ind_p only stores the variants that aren't in any cs
            ind_p <- setdiff(ind_p, rd$sets$cs[[cset]])

            # append variant info
            i <- poswhighestpip
            variant_info[[length(variant_info) + 1]] <- list(rd$chr[i], rd$pos[i], rd$ref[i], rd$alt[i], rid, cset, rd$pip[i])

            table <- paste0(table, "| ", rd$chr[i], " | ", rd$pos[i], " | ", rd$ref[i], " | ", rd$alt[i], " | ", rid, " | ", cset, " | ", sprintf("%.2f", rd$pip[i]), " |  \n")

            sumpips <- sumpips + sum(rd$pip[rd$sets$cs[[cset]]])
            sumvars <- sumvars + length(rd$sets$cs[[cset]])
          } else { # if we have only one variant in the cs
            i <- rd$sets$cs[[cset]]

            # we make sure that ind_p only stores the variants that aren't in any cs
            ind_p <- setdiff(ind_p, i)

            # append variant info
            variant_info[[length(variant_info) + 1]] <- list(rd$chr[i], rd$pos[i], rd$ref[i], rd$alt[i], rid, cset, rd$pip[i])

            table <- paste0(table, "| ", rd$chr[i], " | ", rd$pos[i], " | ", rd$ref[i], " | ", rd$alt[i], " | ", rid, " | ", cset, " | ", sprintf("%.2f", rd$pip[i]), " |  \n")

            sumpips <- sumpips + rd$pip[i]
            sumvars <- sumvars + 1
          }
        }

        text_temp <- paste0(text_temp, "- Total number of variants: ", length(rd$pip), "\n")
        text_temp <- paste0(text_temp, "- Expected number of causal variants: ", sprintf("%.2f", sumpips), "\n")
        text_temp <- paste0(text_temp, "- Number of variants with PIP > ", ${pip_cutoff}, " and not in any CS: ", length(ind_p), "\n\n")
        text_temp <- paste0(text_temp, tbl_header, table, sep)

        if (num_csets[reg_i] > 1) {
          text_temp <- paste0(text_temp, "#### CORR: Correlation between CS | OLAP: Overlap between CS\n")

          cs <- names(rd$sets$cs)

          corrheader <- "|  |"
          corrbreak <- "| --- |"

          for (i in cs) {
            corrheader <- paste0(corrheader, " CORR ", i, " |")
            corrbreak <- paste0(corrbreak, " --- |")
          }

          corrheader <- paste0(corrheader, "  |")
          corrbreak <- paste0(corrbreak, " --- |")

          for (i in cs) {
            corrheader <- paste0(corrheader, " OLAP ", i, " |")
            corrbreak <- paste0(corrbreak, " --- |")
          }

          corrheader <- paste0(corrheader, "\n")
          corrbreak <- paste0(corrbreak, "\n")

          body <- ""

          for (en in seq_along(cs)) {
            i <- cs[en]
            body <- paste0(body, "| ", i, " |")
            for (j in rd$cscorr[[en]]) {
              body <- paste0(body, " ", sprintf("%.2f", j), " |")
            }
            body <- paste0(body, "  |")
            for (j in names(rd$sets$cs)) {
              body <- paste0(body, " ", length(intersect(rd$sets$cs[[i]], rd$sets$cs[[j]])), " |")
            }
            body <- paste0(body, "\n")
          }

          text_temp <- paste0(text_temp, corrheader, corrbreak, body, sep)
        }

        region_info <- c(region_info, text_temp)
      }
    }

    f <- file(${_output["analysis_summary"]:r}, "w")
    writeLines(paste0(theme, text), f)
    close(f)

    for (i in ind_p) {
      # append variant info
      variant_info[[length(variant_info) + 1]] <- list(rd$chr[i], rd$pos[i], rd$ref[i], rd$alt[i], rid, "None", rd$pip[i])
    }

    df <- do.call(rbind, variant_info)
    colnames(df) <- c("chr", "pos", "ref", "alt", "rid", "cs", "pip")
    write.table(df, ${_output["variants_csv"]:r}, sep = "\t", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate analysis report: HTML file, and optionally PPTX file
[SuSiE_slides_2]
output: f"{_input['analysis_summary']:n}.html"
sh: container=container_marp, expand = "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout', entrypoint = entrypoint
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:a} \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files
    node /opt/marp/.cli/marp-cli.js ${_input['analysis_summary']} -o ${_output:an}.pptx \
        --title '${region_file:bnn} fine mapping analysis' \
        --allow-local-files

In [ ]:
sos run pipeline/mnm_regression.ipynb fsusie     --name protocol_example_ATAC    \
--genoFile /hpc/users/sunh14/snmulti_QTL/output/Genotype/snmulti_QTL_analysis_psychAD_ADSP.leftnorm.bcftools_qc.plink_qc.genotype_by_chrom_files.txt   \
--phenoFile output/hpc/users/sunh14/snmulti_QTL/output/caQTL/data_preprocessing/Astro.EC/phenotype_data/pb4dreamlet_subclass_ATAC.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.bed.gz.extended_TADB.bed.16_pheno_per_region.region_list   \
--covFile /hpc/users/sunh14/snmulti_QTL/output/caQTL/data_preprocessing/Astro.EC/covariates/pb4dreamlet_subclass_ATAC.Astro.EC.low_expression_filtered.normalized.log2cpm.gct.cov_pca.Marchenko_PC.gz       \
--container ~/xqtl-protocol-container/pecotmr.sif \
--cis-window 1  --phenotype-names trait_A    --region_name TADB_1189 \
--max_scale 6 --min_purity 1 --susie_top_pc 1 --twas_cv_folds 2  --max_L 2 -n